In [1]:
%cd ../

/mnt/kireev/pycharm-deploy/vtb


In [2]:
import pickle
import random

In [3]:
from glob import glob

In [4]:
from itertools import chain

In [5]:
import numpy as np
import pandas as pd

In [6]:
import torch
import pytorch_lightning as pl

In [7]:
from pyhocon import ConfigFactory

In [8]:
import matplotlib.pyplot as plt

In [9]:
from dltranz.data_load.iterable_processing.category_size_clip import CategorySizeClip
from dltranz.data_load import augmentation_chain
from dltranz.data_load.augmentations.seq_len_limit import SeqLenLimit
from dltranz.data_load.augmentations.random_slice import RandomSlice

from dltranz.seq_encoder import create_encoder

from dltranz.metric_learn.sampling_strategies import get_sampling_strategy
from dltranz.metric_learn.losses import get_loss

from dltranz.tb_interface import get_scalars

In [10]:
from vtb_code.data import PairedDataset, paired_collate_fn, PairedZeroDataset, DropDuplicate
from vtb_code.metrics import PrecisionK, MeanReciprocalRankK, ValidationCallback

In [11]:
FOLD_ID = 1

In [12]:
fold_id_test = FOLD_ID

In [13]:
folds_count = len(glob('data/train_matching_*.csv'))
folds_count

6

In [14]:
# fold_id_valid = np.random.choice([i for i in range(folds_count) if i != fold_id_test], size=1)[0]
fold_id_valid = (fold_id_test + 1) % folds_count
fold_id_valid

2

In [15]:
df_matching_train = pd.concat([pd.read_csv(f'data/train_matching_{i}.csv')
                              for i in range(folds_count) 
                              if i not in (fold_id_test, fold_id_valid)])
df_matching_valid = pd.read_csv(f'data/train_matching_{fold_id_valid}.csv')
df_matching_test = pd.read_csv(f'data/train_matching_{fold_id_test}.csv')

In [16]:
[len(df) for df in [df_matching_train, df_matching_valid, df_matching_test]]

[11721, 2930, 2930]

In [17]:
%%time
with open(f'data/features_f{FOLD_ID}.pickle', 'rb') as f:
    (
        features_trx_train,
        features_trx_valid,
        features_trx_test,
        features_trx_puzzle,
        features_click_train,
        features_click_valid,
        features_click_test,
        features_click_puzzle,
    ) = pickle.load(f)

CPU times: user 9.01 s, sys: 3.45 s, total: 12.5 s
Wall time: 12.5 s


# Preetrain

In [ ]:
train_dl_mlm_trx = torch.utils.data.DataLoader(
    PairedDataset(
        np.sort(np.array(list(features_trx_train.keys()))).reshape(-1, 1),
        data=[features_trx_train],
        augmentations=[augmentation_chain(
            DropDuplicate('mcc_code', col_new_cnt='c_cnt'), 
            RandomSlice(32, 128)
        )],
        n_sample=1,
    ),
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=12,
    batch_size=128,
    persistent_workers=True,
)

valid_dl_mlm_trx = torch.utils.data.DataLoader(
    PairedDataset(
        np.sort(np.array(list(features_trx_test.keys()))).reshape(-1, 1),
        data=[features_trx_test],
        augmentations=[augmentation_chain(
            DropDuplicate('mcc_code', col_new_cnt='c_cnt'), 
            RandomSlice(32, 128)
        )],
        n_sample=1,
    ),
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=12,
    batch_size=128,
    persistent_workers=True,
)

In [ ]:
train_dl_mlm_click = torch.utils.data.DataLoader(
    PairedDataset(
        np.sort(np.array(list(features_click_train.keys()))).reshape(-1, 1),
        data=[features_click_train],
        augmentations=[augmentation_chain(
            DropDuplicate('cat_id', col_new_cnt='c_cnt'), 
            RandomSlice(32, 128)
        )],
        n_sample=1,
    ),
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=12,
    batch_size=128,
    persistent_workers=True,
)

valid_dl_mlm_click = torch.utils.data.DataLoader(
    PairedDataset(
        np.sort(np.array(list(features_click_test.keys()))).reshape(-1, 1),
        data=[features_click_test],
        augmentations=[augmentation_chain(
            DropDuplicate('cat_id', col_new_cnt='c_cnt'), 
            RandomSlice(32, 128)
        )],
        n_sample=1,
    ),
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=12,
    batch_size=128,
    persistent_workers=True,
)

In [ ]:
v = []
for batch in train_dl_mlm_trx:
    v.append(batch[0][0].payload['transaction_amt'][batch[0][0].seq_len_mask.bool()])
v = torch.cat(v)

trx_amnt_quantiles = torch.quantile(torch.unique(v), torch.linspace(0, 1, 100))
trx_amnt_quantiles

In [18]:
from dltranz.trx_encoder import TrxEncoder, PaddedBatch

In [19]:
from vtb_code.models import MeanLoss

In [20]:
class CustomTrxTransform(torch.nn.Module):
    def __init__(self, trx_amnt_quantiles):
        super().__init__()
        self.trx_amnt_quantiles = torch.nn.Parameter(trx_amnt_quantiles, requires_grad=False)
        
    def forward(self, x):
        x.payload['transaction_amt_q'] = torch.bucketize(x.payload['transaction_amt'], self.trx_amnt_quantiles) + 1
        return x
    
class CustomClickTransform(torch.nn.Module):
    def forward(self, x):
#         x.payload['cat_id'] = torch.clamp(x.payload['cat_id'], 0, 300)
#         x.payload['level_0'] = torch.clamp(x.payload['level_0'], 0, 200)
#         x.payload['level_1'] = torch.clamp(x.payload['level_1'], 0, 200)
#         x.payload['level_2'] = torch.clamp(x.payload['level_2'], 0, 200)
#         x.payload['c_cnt_clamp'] = torch.clamp(x.payload['c_cnt'], 0, 20).int()
        return x

In [21]:
class DateFeaturesTransform(torch.nn.Module):
    def forward(self, x):
        et = x.payload['event_time'].int()
        et_day = et.div(24 * 60 * 60, rounding_mode='floor').int()
        x.payload['hour'] = et.div(60 * 60, rounding_mode='floor') % 24 + 1
        x.payload['weekday'] = et.div(60 * 60 * 24, rounding_mode='floor') % 7 + 1
        x.payload['day_diff'] = torch.clamp(torch.diff(et_day, prepend=et_day[:, :1], dim=1), 0, 14)
        return x

In [22]:
class PBLinear(torch.nn.Linear):
    def forward(self, x: PaddedBatch):
        return PaddedBatch(super().forward(x.payload), x.seq_lens)

In [23]:
class PBL2Norm(torch.nn.Module):
    def __init__(self, beta):    
        super().__init__()
        self.beta = beta
    
    def forward(self, x):
        return PaddedBatch(self.beta * x.payload / (x.payload.pow(2).sum(dim=-1, keepdim=True) + 1e-9).pow(0.5), 
                           x.seq_lens)

In [24]:
from vtb_code.data import frequency_encoder

In [25]:
class MLMPretrainModule(pl.LightningModule):
    def __init__(self, data_type, params,
                 lr, weight_decay,
                 max_lr, pct_start, total_steps,
                ):
        super().__init__()
        self.save_hyperparameters()
        
        common_trx_size = params['common_trx_size']
        self.seq_encoder = None
        
        self.token_mask = torch.nn.Parameter(torch.randn(1, 1, common_trx_size), requires_grad=True)
        self.transf = torch.nn.TransformerEncoder(
            encoder_layer=torch.nn.TransformerEncoderLayer(
                d_model=common_trx_size,
                nhead=params['transf.nhead'],
                dim_feedforward=params['transf.dim_feedforward'],
                dropout=params['transf.dropout'],
                batch_first=True,
            ),
            num_layers=params['transf.num_layers'], 
            norm=torch.nn.LayerNorm(common_trx_size) if params['transf.norm'] else None,
        )
        
        if params['transf.use_pe']:
            self.pe = torch.nn.Parameter(self.get_pe(), requires_grad=False)
        else:
            self.pe = None
        self.padding_mask = torch.nn.Parameter(torch.tensor([True, False]).bool(), requires_grad=False)

        self.train_mlm_loss_all = MeanLoss(compute_on_step=False)
        self.valid_mlm_loss_all = MeanLoss(compute_on_step=False)
        self.train_mlm_loss_self = MeanLoss(compute_on_step=False)
        self.valid_mlm_loss_self = MeanLoss(compute_on_step=False)
        
    def get_pe(self):
        max_len = self.hparams.params['transf.max_len']
        H = self.hparams.params['common_trx_size']
        f = 2 * np.pi * torch.arange(max_len).view(1, -1, 1) / \
        torch.exp(torch.linspace(*np.log([4, max_len]), H // 2)).view(1, 1, -1)
        return torch.cat([torch.sin(f), torch.cos(f)], dim=2)
        
    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optim,
            max_lr=self.hparams.max_lr,
            total_steps=self.hparams.total_steps,
            pct_start=self.hparams.pct_start,
            anneal_strategy='cos',
            cycle_momentum=False,
            div_factor=25.0,
            final_div_factor=10000.0,
            three_phase=True,
        )
        scheduler = {'scheduler': scheduler, 'interval': 'step'}
        return [optim], [scheduler]
            
    def get_mask(self, x: PaddedBatch):
        return torch.bernoulli(x.seq_len_mask.float() * self.hparams.params['mlm.replace_proba']).bool()
        
    def mask_x(self, x: PaddedBatch, mask):
        return torch.where(mask.unsqueeze(2).expand_as(x.payload), 
                           self.token_mask.expand_as(x.payload), x.payload)
        
    def get_neg_ix(self, mask, neg_type):
        """Sample from predicts, where `mask == True`, without self element.
        For `neg_type='all'` - sample from predicted tokens from batch
        For `neg_type='self'` - sample from predicted tokens from row
        """
        if neg_type == 'all':
            mn = mask.float().view(1, -1) - \
                torch.eye(mask.numel(), device=mask.device)[mask.flatten()]
            neg_ix = torch.multinomial(mn, self.hparams.params['mlm.neg_count_all'])
            b_ix = neg_ix.div(mask.size(1), rounding_mode='trunc')
            neg_ix = neg_ix % mask.size(1)
            return b_ix, neg_ix
        if neg_type == 'self':
            mask_ix = mask.nonzero(as_tuple=False)
            one_pos = torch.eye(mask.size(1), device=mask.device)[mask_ix[:, 1]]
            mn = mask[mask_ix[:, 0]].float() - one_pos
            mn = mn + 1e-9 * (1 - one_pos)
            neg_ix = torch.multinomial(mn, self.hparams.params['mlm.neg_count_self'], replacement=True)
            b_ix = mask_ix[:, 0].view(-1, 1).expand_as(neg_ix)
            return b_ix, neg_ix
        raise AttributeError(f'Unknown neg_type: {neg_type}')
    
    def sentence_encoding(self, x: PaddedBatch):
        return None
        
    def mlm_loss(self, x: PaddedBatch, neg_type, x_orig: PaddedBatch):
        mask = self.get_mask(x)
        masked_x = self.mask_x(x, mask)
        B, T, H = masked_x.size()
        
        if self.pe is not None:
            if self.training:
                start_pos = np.random.randint(0, self.hparams.params['transf.max_len'] - T, 1)[0]
            else:
                start_pos = 0
            pe = self.pe[:, start_pos:start_pos + T]
            masked_x = masked_x + pe
            
        se = self.sentence_encoding(x_orig)
        if se is not None:
            masked_x = masked_x + se

        out = self.transf(masked_x, src_key_padding_mask=self.padding_mask[x.seq_len_mask])
        
        if self.pe is not None:
            out = out - pe
        if se is not None:
            out = out - se

        target = x.payload[mask].unsqueeze(1)  # N, 1, H
        predict = out[mask].unsqueeze(1) # N, 1, H
        neg_ix = self.get_neg_ix(mask, neg_type)
        negative = out[neg_ix[0], neg_ix[1]]  # N, nneg, H
        out_samples = torch.cat([predict, negative], dim=1)
        probas = torch.softmax((target * out_samples).sum(dim=2), dim=1)
        loss = -torch.log(probas[:, 0])
        return loss
    
    def training_step(self, batch, batch_idx):
        (x_trx, _), = batch
        
        z_trx = self.seq_encoder(x_trx)  # PB: B, T, H
        
        loss_mlm = self.mlm_loss(z_trx, neg_type='all', x_orig=x_trx)
        self.train_mlm_loss_all(loss_mlm)
        loss_mlm_all = loss_mlm.mean()
        self.log(f'loss/mlm_{self.hparams.data_type}', loss_mlm_all)

        loss_mlm = self.mlm_loss(z_trx, neg_type='self', x_orig=x_trx)
        self.train_mlm_loss_self(loss_mlm)
        loss_mlm_self = loss_mlm.mean()
        self.log(f'loss/mlm_{self.hparams.data_type}_self', loss_mlm_self)
        
        return loss_mlm_all + loss_mlm_self

    def validation_step(self, batch, batch_idx):
        (x_trx, _), = batch
        z_trx = self.seq_encoder(x_trx)  # PB: B, T, H
        
        loss_mlm = self.mlm_loss(z_trx, neg_type='all', x_orig=x_trx)
        self.valid_mlm_loss_all(loss_mlm)
        
        loss_mlm = self.mlm_loss(z_trx, neg_type='self', x_orig=x_trx)
        self.valid_mlm_loss_self(loss_mlm)
        
    def training_epoch_end(self, _):
        self.log(f'metrics/train_{self.hparams.data_type}_mlm', self.train_mlm_loss_all, prog_bar=False)
        self.log(f'metrics/train_{self.hparams.data_type}_mlm_self', self.train_mlm_loss_self, prog_bar=False)
        
    def validation_epoch_end(self, _):
        self.log(f'metrics/valid_{self.hparams.data_type}_mlm', self.valid_mlm_loss_all, prog_bar=True)
        self.log(f'metrics/valid_{self.hparams.data_type}_mlm_self', self.valid_mlm_loss_self, prog_bar=True)

        
class MLMPretrainModuleTrx(MLMPretrainModule):
    def __init__(self,
                 trx_amnt_quantiles, 
                 params,
                 lr, weight_decay,
                 max_lr, pct_start, total_steps,
                ):
        super().__init__(data_type='trx',
                         params=params,
                         lr=lr, weight_decay=weight_decay,
                         max_lr=max_lr, pct_start=pct_start, total_steps=total_steps,
                        )
        self.save_hyperparameters()
        
        common_trx_size = self.hparams.params['common_trx_size']
        t = TrxEncoder(self.hparams.params['trx_seq.trx_encoder'])
        self.seq_encoder = torch.nn.Sequential(
            CustomTrxTransform(trx_amnt_quantiles=trx_amnt_quantiles),
            DateFeaturesTransform(),
            t, PBLinear(t.output_size, common_trx_size),
            PBL2Norm(self.hparams.params['mlm.beta']),
        )
        

class MLMPretrainModuleClick(MLMPretrainModule):
    def __init__(self, params,
                 lr, weight_decay,
                 max_lr, pct_start, total_steps,
                ):
        super().__init__(data_type='click',
                         params=params,
                         lr=lr, weight_decay=weight_decay,
                         max_lr=max_lr, pct_start=pct_start, total_steps=total_steps,
                        )
        self.save_hyperparameters()
        
        common_trx_size = self.hparams.params['common_trx_size']
        t = TrxEncoder(self.hparams.params['click_seq.trx_encoder'])
        self.seq_encoder = torch.nn.Sequential(
            CustomClickTransform(),
            DateFeaturesTransform(),
            t, PBLinear(t.output_size, common_trx_size),
            PBL2Norm(self.hparams.params['mlm.beta']),
        )
        
#         self.se = torch.nn.Embedding(7, common_trx_size, padding_idx=0)

#     def sentence_encoding(self, x: PaddedBatch):
#         se = torch.stack([frequency_encoder(v, m.bool())
#                           for v, m in zip(x.payload['new_uid'], x.seq_len_mask)], dim=0).clamp(None, 6)
#         return self.se(se)
    

In [ ]:
config = ConfigFactory.parse_string('''
    common_trx_size: 256
    transf: {
        nhead: 4
        dim_feedforward: 1024
        dropout: 0.1
        num_layers: 3
        norm: false
        max_len: 6000
        use_pe: true
    }
    mlm: {
        replace_proba: 0.1
        neg_count_all: 64
        neg_count_self: 8
        beta: 10
    }
    trx_seq: {
        trx_encoder: {
          use_batch_norm_with_lens: false
          norm_embeddings: false,
          embeddings_noise: 0.000,
          embeddings: {
            mcc_code: {in: 350, out: 64},
            currency_rk: {in: 10, out: 4}
            transaction_amt_q: {in: 110, out: 8}
            
            hour: {in: 30, out: 16}
            weekday: {in: 10, out: 4}
            day_diff: {in: 15, out: 8}
          },
          numeric_values: {
            transaction_amt: identity
            c_cnt: log
          }
          was_logified: false
          log_scale_factor: 1.0
        },
    }
    click_seq: {
        trx_encoder: {
          use_batch_norm_with_lens: false
          norm_embeddings: false,
          embeddings_noise: 0.000,
          embeddings: {
            cat_id: {in: 400, out: 64},
            level_0: {in: 400, out: 16}
            level_1: {in: 400, out: 8}
            level_2: {in: 400, out: 4}
            
            hour: {in: 30, out: 16}
            weekday: {in: 10, out: 4}
            day_diff: {in: 15, out: 8}
          },
          numeric_values: {
            c_cnt: log
          }
          was_logified: false
          log_scale_factor: 1.0
        },
    }
''')

mlm_model_trx = MLMPretrainModuleTrx(
    params=config,                     
    lr=0.001, weight_decay=0,
    max_lr=0.001, pct_start=9000 / 2 / 10000, total_steps=10000,
    trx_amnt_quantiles=trx_amnt_quantiles,
)
mlm_model_click = MLMPretrainModuleClick(
    params=config,                     
    lr=0.001, weight_decay=0,
    max_lr=0.001, pct_start=9000 / 2 / 10000, total_steps=10000,
)


In [ ]:
trainer = pl.Trainer(
    gpus=[0],
    max_steps=10000,
    enable_progress_bar=False,
    callbacks=[
        pl.callbacks.LearningRateMonitor(),
        pl.callbacks.ModelCheckpoint(
            every_n_train_steps=2000, save_top_k=-1,
        ),
    ]
)
model_version_trx = trainer.logger.version
print('baseline all:  {:.3f}'.format(np.log(mlm_model_trx.hparams.params['mlm.neg_count_all'] + 1)))
print('baseline self: {:.3f}'.format(np.log(mlm_model_trx.hparams.params['mlm.neg_count_self'] + 1)))
print(f'version = {model_version_trx}')
trainer.fit(mlm_model_trx, train_dl_mlm_trx, valid_dl_mlm_trx)
print('done')

In [ ]:
trainer = pl.Trainer(
    gpus=[0],
    max_steps=10000,
    enable_progress_bar=False,
    callbacks=[
        pl.callbacks.LearningRateMonitor(),
        pl.callbacks.ModelCheckpoint(
            every_n_train_steps=2000, save_top_k=-1,
        ),
    ]
)
model_version_click = trainer.logger.version
print('baseline all:  {:.3f}'.format(np.log(mlm_model_click.hparams.params['mlm.neg_count_all'] + 1)))
print('baseline self: {:.3f}'.format(np.log(mlm_model_click.hparams.params['mlm.neg_count_self'] + 1)))
print(f'version = {model_version_click}')
trainer.fit(mlm_model_click, train_dl_mlm_click, valid_dl_mlm_click)
print('done')

# Use pretrained

In [26]:
model_version_trx = 148
model_version_click = 149

In [27]:
"""
42, 43 - 256 without norm

70, 71 - 256 without L2Norm and beta=10, neg_count_all=16

72, 74 - 512 without L2Norm and beta=10, neg_count_all=16

73, 75 - 512 without L2Norm and beta=10, neg_count_all=64

_,  117 - 512 without L2Norm and beta=10, neg_count_all=64, with sentence_encoding
"""


mlm_model_trx = MLMPretrainModuleTrx.load_from_checkpoint(
    'lightning_logs/version_148/checkpoints/epoch=108-step=9999.ckpt')  # 42
mlm_model_click = MLMPretrainModuleClick.load_from_checkpoint(
    'lightning_logs/version_149/checkpoints/epoch=129-step=9999.ckpt')  # 43

In [28]:
# mlm_model_trx.freeze()
# mlm_model_click.freeze()

In [29]:
from dltranz.seq_encoder.utils import NormEncoder

In [30]:
from dltranz.trx_encoder import TrxEncoder, PaddedBatch
from dltranz.seq_encoder.rnn_encoder import RnnEncoder
from dltranz.seq_encoder.utils import LastStepEncoder

In [31]:
class L2Scorer(torch.nn.Module):
    def forward(self, x):
        B, H = x.size()
        a, b =x[:, :H // 2], x[:, H // 2:]
        return -(a - b).pow(2).sum(dim=1)

In [32]:
class PBLayerNorm(torch.nn.LayerNorm):
    def forward(self, x: PaddedBatch):
        return PaddedBatch(super().forward(x.payload), x.seq_lens)

In [33]:
class PairedModule(pl.LightningModule):
    def __init__(self, params, k,
                 lr, weight_decay,
                 max_lr, pct_start, total_steps,
                 beta, neg_count,
                ):
        super().__init__()
        self.save_hyperparameters()
        
        common_trx_size = mlm_model_trx.hparams.params['common_trx_size']
        self.rnn_enc =  torch.nn.Sequential(
            RnnEncoder(common_trx_size, params['rnn']), 
            LastStepEncoder(),
#             NormEncoder(),
        )
        self._seq_encoder_trx = torch.nn.Sequential(
            mlm_model_trx.seq_encoder,
            PBLayerNorm(common_trx_size),
        )
        self._seq_encoder_click = torch.nn.Sequential(
            mlm_model_click.seq_encoder,
            PBLayerNorm(common_trx_size),
        )
        self.mlm_model_click = mlm_model_click
        
        self.cls = torch.nn.Sequential(
            L2Scorer(),
        )

        self.train_precision = PrecisionK(k=k, compute_on_step=False)
        self.train_mrr = MeanReciprocalRankK(k=k, compute_on_step=False)
        self.valid_precision = PrecisionK(k=k, compute_on_step=False)
        self.valid_mrr = MeanReciprocalRankK(k=k, compute_on_step=False)
        
    def seq_encoder_trx(self, x):
        x = self._seq_encoder_trx(x)
        return self.rnn_enc(x)
    
    def seq_encoder_click(self, x_orig):
        x = self._seq_encoder_click(x_orig)
#         x = PaddedBatch(
#             x.payload + self.mlm_model_click.sentence_encoding(x_orig),
#             x.seq_lens,
#         )
        return self.rnn_enc(x)

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, weight_decay=self.hparams.weight_decay)
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer=optim,
            max_lr=self.hparams.max_lr,
            total_steps=self.hparams.total_steps,
            pct_start=self.hparams.pct_start,
            anneal_strategy='cos',
            cycle_momentum=False,
            div_factor=25.0,
            final_div_factor=10000.0,
            three_phase=True,
        )
        scheduler = {'scheduler': scheduler, 'interval': 'step'}
        return [optim], [scheduler]

    def loss_fn_p(self, embeddings, labels, ref_emb, ref_labels):
        beta = self.hparams.beta
        neg_count = self.hparams.neg_count
        
        pos_ix = (labels.view(-1, 1) == ref_labels.view(1, -1)).nonzero(as_tuple=False)
        pos_labels = labels[pos_ix[:, 0]]
        neg_w = ((pos_labels.view(-1, 1) != ref_labels.view(1, -1))).float()
        neg_ix = torch.multinomial(neg_w, neg_count - 1)
        all_ix = torch.cat([pos_ix[:, [1]], neg_ix], dim=1)
        logits = -(embeddings[pos_ix[:, [0]]] - ref_emb[all_ix]).pow(2).sum(dim=2)
        logits = logits * beta
        logs = -torch.log(torch.softmax(logits, dim=1))[:, 0]
#         logs = torch.relu(logs + np.log(0.1))
        return logs.mean()

    def training_step(self, batch, batch_idx):
        # pairs
        x_trx, l_trx, m_trx, x_click, l_click, m_click = batch
        z_trx = self.seq_encoder_trx(x_trx)  # B, H
        z_click = self.seq_encoder_click(x_click)  # B, H
        loss_pt = self.loss_fn_p(embeddings=z_trx, labels=l_trx, ref_emb=z_click, ref_labels=l_click)
        self.log('loss/loss_pt', loss_pt)
        
        loss_pc = self.loss_fn_p(embeddings=z_click, labels=l_click, ref_emb=z_trx, ref_labels=l_trx)
        self.log('loss/loss_pc', loss_pc)
       
        with torch.no_grad():
            out = -(z_trx.unsqueeze(1) - z_click.unsqueeze(0)).pow(2).sum(dim=2)
            out = out[m_trx == 0][:, m_click == 0]
            T, C = out.size()
            assert T == C
            n_samples = z_trx.size(0) // (l_trx.max().item() + 1)
            for i in range(n_samples):
                l2 = out[i::n_samples, i::n_samples]
                self.train_precision(l2)
                self.train_mrr(l2)
        
        return loss_pt + 0.1 * loss_pc  #  loss_pc 

    def training_epoch_end(self, _):
        self.log('train_metrics/precision', self.train_precision, prog_bar=True)
        self.log('train_metrics/mrr', self.train_mrr, prog_bar=True)


In [34]:
batch_size = 128
train_dl = torch.utils.data.DataLoader(
    PairedZeroDataset(
        pd.concat([df_matching_train, df_matching_test], axis=0)[lambda x: x['rtk'].ne('0')].values,
        data=[
            dict(chain(features_trx_train.items(), features_trx_test.items())),
            dict(chain(features_click_train.items(), features_click_test.items())),
        ],
        augmentations=[
            augmentation_chain(DropDuplicate('mcc_code', col_new_cnt='c_cnt'), RandomSlice(32, 1024)),  # 1024
            augmentation_chain(DropDuplicate('cat_id', col_new_cnt='c_cnt'), RandomSlice(64, 2048)),  # 2048
        ],
        n_sample=2,
    ),
    collate_fn=PairedZeroDataset.collate_fn,
    drop_last=True,
    shuffle=True,
    num_workers=24,
    batch_size=batch_size,
    persistent_workers=True,
)

valid_dl_trx = torch.utils.data.DataLoader(
    PairedDataset(
        np.sort(df_matching_valid['bank'].unique()).reshape(-1, 1), 
        data=[
            features_trx_valid,
        ],
        augmentations=[
            augmentation_chain(DropDuplicate('mcc_code', col_new_cnt='c_cnt'), SeqLenLimit(2000)),  # 2000
        ],
        n_sample=1,
    ),
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=4,
    batch_size=128,
    persistent_workers=True,
)

valid_dl_click = torch.utils.data.DataLoader(
    PairedDataset(
        np.sort(df_matching_valid[lambda x: x['rtk'].ne('0')]['rtk'].unique()).reshape(-1, 1),
        data=[
            features_click_valid,
        ],
        augmentations=[
            augmentation_chain(DropDuplicate('cat_id', col_new_cnt='c_cnt'), SeqLenLimit(5000)),  # 5000
        ],
        n_sample=1,
    ),
    collate_fn=paired_collate_fn,
    shuffle=False,
    num_workers=4,
    batch_size=128,
    persistent_workers=True,
)

In [35]:
sup_model = PairedModule(
    ConfigFactory.parse_string('''
    common_trx_size: 128
    rnn: {
      type: gru,
      hidden_size: 256,
      bidir: false,
      trainable_starter: static
    }
'''),                     
    k=100 * batch_size // 3000,
    lr=0.0022, weight_decay=0,
    max_lr=0.0018, pct_start=1100 / 6000, total_steps=6000,
    beta=0.2 / 1.4, neg_count=120,
)


In [36]:
class ValidationCallback(pl.Callback):
    def __init__(self, v_trx, v_click, target, device, device_main, k=100, batch_size=1024):
        self.v_trx = v_trx
        self.v_click = v_click
        self.target = target
        self.device = device
        self.device_main = device_main
        self.k = k
        self.batch_size = batch_size

    def on_train_epoch_end(self, trainer, pl_module):
        was_traning = False
        if pl_module.training:
            pl_module.eval()
            was_traning = True

        pl_module.to(self.device)
        with torch.no_grad():
            z_trx = []
            for ((x_trx, _),) in self.v_trx:
                z_trx.append(pl_module.seq_encoder_trx(x_trx.to(self.device)))
            z_trx = torch.cat(z_trx, dim=0)
            z_click = []
            for ((x_click, _),) in self.v_click:
                z_click.append(pl_module.seq_encoder_click(x_click.to(self.device)))
            z_click = torch.cat(z_click, dim=0)

            T = z_trx.size(0)
            C = z_click.size(0)
            device = z_trx.device
            ix_t = torch.arange(T, device=device).view(-1, 1).expand(T, C).flatten()
            ix_c = torch.arange(C, device=device).view(1, -1).expand(T, C).flatten()

            z_out = []
            for i in range(0, len(ix_t), self.batch_size):
                z_pairs = torch.cat([
                    z_trx[ix_t[i:i + self.batch_size]],
                    z_click[ix_c[i:i + self.batch_size]],
                ], dim=1)
                z_out.append(pl_module.cls(z_pairs).unsqueeze(1))
            z_out = torch.cat(z_out, dim=0).view(T, C)

            precision, mrr, r1 = self.logits_to_metrics(z_out)

            pl_module.log('valid_full_metrics/precision', precision, prog_bar=True)
            pl_module.log('valid_full_metrics/mrr', mrr, prog_bar=False)
            pl_module.log('valid_full_metrics/r1', r1, prog_bar=False)

        pl_module.to(self.device_main)
        if was_traning:
            pl_module.train()

    def logits_to_metrics(self, z_out):
        T, C = z_out.size()
        z_ranks = torch.zeros_like(z_out)
        z_ranks[
            torch.arange(T, device=self.device).view(-1, 1).expand(T, C),
            torch.argsort(z_out, dim=1, descending=True),
        ] = torch.arange(C, device=self.device).float().view(1, -1).expand(T, C) + 1
        z_ranks = torch.cat([
            torch.ones(T, device=self.device).float().view(-1, 1),
            z_ranks + 1,
        ], dim=1)
        
        click_uids = np.concatenate([['0'], self.v_click.dataset.pairs[:, 0]])
        true_ranks = z_ranks[
            np.arange(T),
            np.searchsorted(click_uids,
                            self.target.set_index('bank')['rtk'].loc[self.v_trx.dataset.pairs[:, 0]].values)
        ]
        precision = torch.where(true_ranks <= self.k,
                                torch.ones(1, device=self.device), torch.zeros(1, device=self.device)).mean()
        mrr = torch.where(true_ranks <= self.k, 1 / true_ranks, torch.zeros(1, device=self.device)).mean()
        r1 = 2 * mrr * precision / (mrr + precision)
        return precision, mrr, r1


In [37]:
trainer = pl.Trainer(
    gpus=[0],
    max_steps=3000,
    callbacks=[
        pl.callbacks.LearningRateMonitor(),
        pl.callbacks.ModelCheckpoint(
            every_n_train_steps=1000, save_top_k=-1,
        ),
        ValidationCallback(valid_dl_trx, valid_dl_click, df_matching_valid,
                           torch.device('cuda:0'), torch.device('cuda:0')),
    ]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(sup_model, train_dl)  # valid_dl

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name               | Type                   | Params
--------------------------------------------------------------
0 | rnn_enc            | Sequential             | 395 K 
1 | _seq_encoder_trx   | Sequential             | 52.0 K
2 | _seq_encoder_click | Sequential             | 69.2 K
3 | mlm_model_click    | MLMPretrainModuleClick | 4.0 M 
4 | cls                | Sequential             | 0     
5 | train_precision    | PrecisionK             | 0     
6 | train_mrr          | MeanReciprocalRankK    | 0     
7 | valid_precision    | PrecisionK             | 0     
8 | valid_mrr          | MeanReciprocalRankK    | 0     
--------------------------------------------------------------
2.9 M     Trainable params
1.5 M     Non-trainable params
4.4 M     Total params
17.687    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
"""
0: v78
1: v79
2: v80
3: v81
4: v82
5: v83
"""

In [26]:
df_m = get_scalars('lightning_logs/').set_index('version').loc[[f'version_{i}' for i in [78, 79, 80, 81, 82, 83]]]

# df = df_m[lambda x: x['tag'].str.startswith('train_metrics')] \
# .pivot(index='step', columns='tag', values='value')
# _, axs = plt.subplots(2, 1, figsize=(16, 15))
# for col, ax in zip(df.columns, axs):
#     df[col].plot(ax=ax, title=col, grid=True)
# plt.show()

# df = df_m[lambda x: x['tag'].str.startswith('valid_full_metrics')] \
# .pivot(index='step', columns='tag', values='value')
# _, axs = plt.subplots(3, 1, figsize=(16, 18))
# for col, ax in zip(df.columns, axs):
#     df[col].plot(ax=ax, title=col, grid=True)
# plt.show()

In [27]:
df_m[lambda x: x['tag'].str.startswith('valid_full_metrics')] \
.pivot_table(index='tag', columns='version', values='value', aggfunc=lambda x: x[-1]).round(4)

version,version_78,version_79,version_80,version_81,version_82,version_83
tag,,,,,,
valid_full_metrics/mrr,0.1972,0.1955,0.1959,0.1936,0.1946,0.1957
valid_full_metrics/precision,0.5174,0.5014,0.5038,0.4877,0.4966,0.4971
valid_full_metrics/r1,0.2856,0.2813,0.2821,0.2772,0.2796,0.2808


In [28]:
df_m[lambda x: x['tag'].str.startswith('valid_full_metrics')] \
.pivot_table(index='tag', columns='version', values='value', aggfunc=lambda x: x[-1]).mean(axis=1).round(4)

tag
valid_full_metrics/mrr          0.1954
valid_full_metrics/precision    0.5007
valid_full_metrics/r1           0.2811
dtype: float64